# Скачивание файлов близкородственной бактерии

In [1]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
y
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [2]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

# Установка програм 

In [3]:
%%capture
!pip install biopython
!apt-get update
!apt-get install ncbi-blast+
!wget https://raw.githubusercontent.com/SpryGorgon/hse22_hw2/main/data/scaffolds.fasta

Прочитать подробнее можно тут https://www.ncbi.nlm.nih.gov/books/NBK279690/ 

# Предсказание рибосомальной рнк

In [4]:
from pprint import pprint
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.SeqRecord import SeqRecord

mil_1_genome = SeqIO.read("/content/T_oleivorans_MIL_1.gbk", "genbank")

In [5]:
records=[]
for feature in mil_1_genome.features:
    if(feature.type=='rRNA' and feature.qualifiers['product'][0][:3] in ['5S ', '16S', '23S']):
        record = records.append(SeqRecord(feature.extract(mil_1_genome.seq)))
        records[-1].id = feature.qualifiers['locus_tag'][0]
        records[-1].description = feature.qualifiers['product'][0]

SeqIO.write(records, "query.fasta", "fasta")

12

In [6]:
# Ваш код
!blastn -query query.fasta -subject scaffolds.fasta -outfmt 6 > blastn.txt
!blastn -query query.fasta -subject scaffolds.fasta -outfmt 7

# BLASTN 2.6.0+
# Query: TOL_0319 16S ribosomal RNA
# Database: User specified sequence set (Input: scaffolds.fasta)
# Fields: query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score
# 3 hits found
TOL_0319	scaffold1_cov232	100.000	1539	0	0	1	1539	2526527	2528065	0.0	2843
TOL_0319	scaffold1_cov232	99.935	1539	1	0	1	1539	846845	845307	0.0	2837
TOL_0319	scaffold32_cov590	100.000	1399	0	0	134	1532	1399	1	0.0	2584
# BLASTN 2.6.0+
# Query: TOL_0322 23S ribosomal RNA
# Database: User specified sequence set (Input: scaffolds.fasta)
# Fields: query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score
# 5 hits found
TOL_0322	scaffold1_cov232	100.000	2887	0	0	1	2887	2528316	2531202	0.0	5332
TOL_0322	scaffold1_cov232	99.965	2887	1	0	1	2887	845056	842170	0.0	5326
TOL_0322	scaffold35_cov581	100.000	1535	0	0	1353	2887	1815	281	0.0	2835
TOL

In [7]:
%%capture
!wget https://raw.githubusercontent.com/SpryGorgon/hse22_hw2/main/data/GENOME.gbk

In [8]:
genome = SeqIO.to_dict(SeqIO.parse("/content/GENOME.gbk", "genbank"))
lines=[]
with open('blastn.txt') as f:
    lines.extend(f.readlines())

for line in lines:
    words = line.split()
    blast_query, blast_subject = words[0], words[1]
    for record in records:
        if (record.id != blast_query) or (blast_subject not in genome):
            continue
        _start, _end, _strand = int(words[8]), int(words[9]), 1 if words[8]<words[9] else -1
        if(_start>_end):
            _start, _end = _end, _start
        feature = SeqFeature(FeatureLocation(_start, _end, _strand), type='rRNA')
        feature.qualifiers['note'] = words[0]
        
        genome[blast_subject].features.append(feature)

pprint([record[1] for record in genome.items()][0].features[-1].qualifiers)

OrderedDict([('note', 'TOL_3255')])


In [9]:
SeqIO.write([record[1] for record in genome.items()], "GENOME_bonus.gbk", "genbank")

69